# Walkthrough Notebook

Notes:

data["geometry"][10] is a good counterexample, on LHS issue of needing to clock going OUT but current method does not.

data["geometry"][257] (Heyroyd) is weird and shows issues with OSM. There is something that SHOULD be a residential area at the top, but the line goes straight through. Just grey "nothing" - no tags

In [ ]:
import urllib

import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd
import plotly.graph_objects as go
import polars as pl
import pyproj
import requests
from brdr.aligner import Aligner
from brdr.enums import OpenbaarDomeinStrategy
from brdr.loader import DictLoader
from matplotlib.widgets import Slider
from shapely import LineString, MultiLineString, MultiPolygon, Polygon
from shapely.ops import unary_union
from shapely.wkt import loads

In [ ]:
datasette_base_url = "https://datasette.planning.data.gov.uk/conservation-area.csv"

query = """
select * 
from entity
"""
encoded_query = urllib.parse.urlencode({"sql": query})

r = requests.get(f"{datasette_base_url}?{encoded_query}", auth=("user", "pass"))

filename = "datasette_data.csv"
with open(filename, "wb") as f_out:
    f_out.write(r.content)

data = pl.read_csv(filename)

In [ ]:
def download_osm_polygons(
    polygon_for_osm,
    osm_tags,
    osm_query_crs,
    plot_crs,
    line_buffer,
):
    # Download Open Street Map polygons
    osm_features_df = gpd.GeoDataFrame(geometry=[], crs=osm_query_crs)
    osm_features_base = ox.features_from_polygon(polygon_for_osm, osm_tags)
    osm_features_df = osm_features_base[
        osm_features_base.geometry.geom_type.isin(
            ["Polygon", "MultiPolygon", "LineString"]
        )
    ]
    # This ensures that LineStrings are also interpreted as v.small polygons - shortcut for all in one solution.
    osm_features_df = osm_features_df.to_crs(plot_crs)
    line_mask = osm_features_df["geometry"].geom_type == "LineString"
    osm_features_df.loc[line_mask, "geometry"] = osm_features_df.loc[
        line_mask, "geometry"
    ].buffer(line_buffer)

    osm_features_df = osm_features_df.buffer(
        0
    )  # Buffer of zero is to ensure it is a GeoPandas series, rather than a df with everything else.

    # Align OSM co-ordinate systems
    print(f"Downloaded {len(osm_features_df)} OSM polygons.")

    # CHECK IF WE NEED THIS!!
    if not osm_features_df.empty:
        if osm_features_df.crs == plot_crs:
            osm_features_proj = osm_features_df
        else:
            osm_features_proj = osm_features_df.to_crs(plot_crs)
    else:
        osm_features_proj = gpd.GeoDataFrame(geometry=[], crs=plot_crs)
    return osm_features_proj

In [ ]:
def process_osm_polygons(osm_features_proj, used_osm_indices):
    reference_geom = None

    if not osm_features_proj.empty:
        geometries_to_combine = None

        if used_osm_indices is None:
            # Here we use ALL downloaded features
            print("Using all polygons.")
            geometries_to_combine = osm_features_proj.geometry
        elif isinstance(used_osm_indices, list) and used_osm_indices:
            # Here we only use features at specific indices
            print(f"Selecting polygons at indices {used_osm_indices}.")
            geometries_to_combine = osm_features_proj.iloc[used_osm_indices].geometry

        # Combine all the geometries we need
        combined_reference = unary_union(geometries_to_combine)
        if not combined_reference.is_empty:
            reference_geom = combined_reference
        else:
            print("Empty osm polygons when combining")
    else:
        print("No OSM polygons downloaded.")
    return reference_geom

In [ ]:
def get_snapped_polygon(
    reference_geom,
    original_proj,
    brdr_distance,
    brdr_strategy,
    brdr_threshold,
    plot_crs,
):
    aligned_df = None
    thematic_geom = original_proj["geometry"].iloc[0]
    if not thematic_geom.is_valid:
        thematic_geom = thematic_geom.buffer(0)

    # alignment logic
    aligner = Aligner(crs=plot_crs)
    loader_thematic = DictLoader({"theme_id_1": thematic_geom})
    aligner.load_thematic_data(loader_thematic)
    loader_reference = DictLoader({"ref_id_1": reference_geom})
    aligner.load_reference_data(loader_reference)

    process_result = aligner.process(
        relevant_distance=brdr_distance,
        od_strategy=brdr_strategy,
        threshold_overlap_percentage=brdr_threshold,
    )

    aligned_geom = process_result["theme_id_1"][brdr_distance]["result"]
    diff_geom = process_result["theme_id_1"][brdr_distance]["result_diff"]
    if not aligned_geom.is_valid:
        aligned_geom = aligned_geom.buffer(0)

    aligned_df = gpd.GeoDataFrame([1], geometry=[aligned_geom], crs=plot_crs)

    if not diff_geom.is_valid:
        diff_geom = diff_geom.buffer(0)

    diff_df = gpd.GeoDataFrame([1], geometry=[diff_geom], crs=plot_crs)

    return aligned_df, diff_df, process_result

In [ ]:
def process_areas(
    original_wkt: str,
    initial_crs: str = "EPSG:4326",
    osm_tags: dict | None = None,
    brdr_distance: float = 20,
    brdr_threshold: float = 10,
    brdr_strategy=OpenbaarDomeinStrategy.SNAP_ONLY_VERTICES,
    plot_crs: str = "EPSG:3857",
    osm_query_crs: str = "EPSG:4326",
    used_osm_indices: list | None = None,
    polygon_detection_buffer: float = 0.00001,
    line_buffer: float = 0.00001,
):

    # Load datasette polygon for area
    original_geom = loads(original_wkt)
    original_df = gpd.GeoDataFrame([1], geometry=[original_geom], crs=initial_crs)
    adding_buffer_df = original_df.to_crs(plot_crs).boundary.buffer(
        polygon_detection_buffer
    )
    # polygon_for_osm = original_df
    # Ensures correct crs
    # df_for_query = original_df.to_crs(osm_query_crs)
    df_for_query = adding_buffer_df.to_crs(osm_query_crs)

    # Need to add a small buffer to ensure that all nearby features are captured. May need testing.
    # polygon_for_osm = df_for_query.geometry.iloc[0].boundary.buffer(polygon_detection_buffer) # Changed .boundary here to avoid getting inner ones
    polygon_for_osm = df_for_query.geometry.iloc[
        0
    ]  # Changed .boundary here to avoid getting inner ones

    # if original_df.crs == plot_crs:
    #     original_proj = original_df
    # else:
    original_proj = original_df.to_crs(plot_crs)

    # polygon_for_osm = polygon_for_osm.buffer(polygon_detection_buffer) # Changed .boundary here to avoid getting inner ones

    # # Download Open Street Map polygons
    try:
        osm_features_proj = download_osm_polygons(
            polygon_for_osm=polygon_for_osm,
            osm_tags=osm_tags,
            osm_query_crs=osm_query_crs,
            plot_crs=plot_crs,
            line_buffer=line_buffer,
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Returning 'None'")
        return None
    # Prep natural features
    reference_geom = process_osm_polygons(osm_features_proj, used_osm_indices)

    # Brdr for snapping polygons
    aligned_df, diff_df, process_result = get_snapped_polygon(
        reference_geom=reference_geom,
        original_proj=original_proj,
        brdr_distance=brdr_distance,
        brdr_strategy=brdr_strategy,
        brdr_threshold=brdr_threshold,
        plot_crs=plot_crs,
    )
    # print(osm_features_proj.explode())
    original_border = original_df["geometry"].to_crs("EPSG:3857")[0]
    new_border = aligned_df["geometry"][0]
    base_features = MultiPolygon(list(osm_features_proj.explode()))
    difference_area = base_features.intersection(diff_df["geometry"])
    difference_area = difference_area.explode()
    difference_area = difference_area[
        difference_area.geometry.geom_type.isin(["Polygon", "MultiPolygon"])
    ]
    difference_area = MultiPolygon(list(difference_area))

    return original_border, new_border, difference_area, base_features

In [ ]:
def polygon_prep(
    polygon,
    transformer,  # See if this needs to be moved inside
):
    # Need to convert from crs to co-ords for this plotly method
    lons = []
    lats = []
    # Shortcut to avoid separate functions
    try:
        if polygon.geom_type == "Polygon":
            polygon = MultiPolygon([polygon])
    except:
        pass
    for poly in polygon.geoms:
        boundary = poly.boundary
        if isinstance(boundary, MultiLineString):
            boundary = boundary.geoms[0]
        x_coords, y_coords = boundary.coords.xy
        lon, lat = transformer.transform(x_coords, y_coords)
        lons.extend(list(lon))
        lats.extend(list(lat))
        # Need to add separator for multiple polygons
        lons.append(None)
        lats.append(None)

    return lons, lats

In [ ]:
def plot_area_with_sliders(
    original_border,
    new_border,
    difference_area,
    diff_rgb,
    base_features,
    base_rgb,
    alpha,
    source_crs="EPSG:3857",
    target_crs="EPSG:4326",
):
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    diff_lons, diff_lats = polygon_prep(difference_area, transformer)
    feature_lons, feature_lats = polygon_prep(base_features, transformer)

    original_lons, original_lats = polygon_prep(original_border, transformer)
    new_lons, new_lats = polygon_prep(new_border, transformer)

    boundary_center = original_border.centroid
    center_lon, center_lat = transformer.transform(boundary_center.x, boundary_center.y)

    diff_fill_colour = f"rgba({diff_rgb[0]}, {diff_rgb[1]}, {diff_rgb[2]}, {alpha})"
    diff_line_colour = f"rgba({diff_rgb[0]}, {diff_rgb[1]}, {diff_rgb[2]}, {0.6})"
    feature_fill_colour = f"rgba({base_rgb[0]}, {base_rgb[1]}, {base_rgb[2]}, {alpha})"
    feature_line_colour = f"rgba({base_rgb[0]}, {base_rgb[1]}, {base_rgb[2]}, {0.6})"

    diff_steps = []
    feature_steps = []
    alphas = np.linspace(0, 1, 11)
    for i, alpha_step in enumerate(alphas):
        alpha_step = round(alpha_step, 2)
        diff_step_color = (
            f"rgba({diff_rgb[0]}, {diff_rgb[1]}, {diff_rgb[2]}, {alpha_step})"
        )
        feature_step_color = (
            f"rgba({base_rgb[0]}, {base_rgb[1]}, {base_rgb[2]}, {alpha_step})"
        )

        diff_step = dict(
            method="restyle",
            args=[{"fillcolor": [diff_step_color]}, [0]],
            label=str(alpha_step),
        )
        diff_steps.append(diff_step)

        feature_step = dict(
            method="restyle",
            args=[{"fillcolor": [feature_step_color]}, [1]],
            label=str(alpha_step),
        )
        feature_steps.append(feature_step)

    diff_sliders = [
        dict(
            active=3,
            currentvalue={"prefix": "Added Area alpha: ", "visible": True},
            pad={"t": 20},
            steps=diff_steps,
        )
    ]

    feature_sliders = [
        dict(
            active=3,
            currentvalue={"prefix": "Base Features alpha: ", "visible": True},
            pad={"t": 120},
            steps=feature_steps,
        )
    ]

    fig = go.Figure()
    fig.add_trace(
        go.Scattermap(
            lon=diff_lons,
            lat=diff_lats,
            mode="lines",
            fill="toself",
            fillcolor=diff_fill_colour,
            line=dict(color=diff_line_colour, width=1),
            name="Concerning Areas",
            showlegend=True,
        )
    )

    fig.add_trace(
        go.Scattermap(
            lon=feature_lons,
            lat=feature_lats,
            mode="lines",
            fill="toself",
            fillcolor=feature_fill_colour,
            line=dict(color=feature_line_colour, width=1),
            name="Base Features",
            showlegend=True,
        )
    )

    fig.add_trace(
        go.Scattermap(
            lon=original_lons,
            lat=original_lats,
            mode="lines",
            fill="none",
            line=dict(color="black", width=3),
            name="Original Boundary",
            showlegend=True,
        )
    )

    fig.add_trace(
        go.Scattermap(
            lon=new_lons,
            lat=new_lats,
            mode="lines",
            fill="none",
            line=dict(color="red", width=1),
            name="New Boundary",
            showlegend=True,
        )
    )

    initial_zoom = 15

    satellite_layer = [
        dict(
            below="traces",
            sourcetype="raster",
            source=[
                "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
            ],
        )
    ]

    map_switch = [
        dict(
            args=[{"map.style": "open-street-map", "map.layers": None}],
            label="OSM",
            method="relayout",
        ),
        dict(
            args=[{"map.style": None, "map.layers": satellite_layer}],
            label="Satellite",
            method="relayout",
        ),
    ]

    fig.update_layout(
        title_text=f"MultiPolygon with Opacity Slider (EPSG:projected)",
        geo_scope="europe",
        map=dict(
            style="open-street-map",
            center=dict(lon=center_lon, lat=center_lat),
            zoom=initial_zoom,
        ),
        showlegend=True,
        margin={"r": 100, "t": 50, "l": 100, "b": 50},
        sliders=diff_sliders + feature_sliders,
        height=800,
        width=1200,
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=map_switch,
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.7,
                xanchor="right",
                y=1.15,
                yanchor="top",
            ),
        ],
    )

    fig.show()

In [ ]:
input_tags = {
    "landuse": ["residential", "farmyard", "farmland", "cemetery", "allotments"],
    # 'landuse': ['farmyard'],
    # 'landuse': ['farmland'],
    # 'highway': ['track'],
    # 'waterway': ['drain'],
    "natural": ["wood"],
    # "highway": ['primary', 'secondary', 'unclassified']
}
input_brdr_distance = 50
input_brdr_threshold = 10
# input_brdr_strategy = OpenbaarDomeinStrategy.SNAP_ONLY_VERTICES
input_brdr_strategy = OpenbaarDomeinStrategy.SNAP_PREFER_VERTICES
# input_brdr_strategy = OpenbaarDomeinStrategy.SNAP_ALL_SIDE
initial_crs = "EPSG:4326"
target_crs = "EPSG:3857"  # CRS for brdr and plotting
target_osm_crs = "EPSG:4326"  # CRS for osm input polygon

areas_tuple = process_areas(
    original_wkt=data["geometry"][866],
    initial_crs=initial_crs,
    osm_tags=input_tags,
    brdr_distance=input_brdr_distance,
    brdr_threshold=input_brdr_threshold,
    brdr_strategy=input_brdr_strategy,
    plot_crs=target_crs,
    osm_query_crs=target_osm_crs,
    used_osm_indices=None,
    line_buffer=10,
    polygon_detection_buffer=1,
)

if areas_tuple:
    original_border, new_border, difference_area, base_features = areas_tuple

In [ ]:
plot_area_with_sliders(
    original_border,
    new_border,
    difference_area,
    (255, 0, 0),
    base_features,
    (0, 0, 255),
    0.3,
    source_crs="EPSG:3857",
    target_crs="EPSG:4326",
)